## scraping of flora online for plant names ##


In [1]:
import wikipediaapi
import pandas as pd
import collections
import re
import requests
from bs4 import BeautifulSoup

In [2]:
# plant name will eventually be an input from csv file

plant_name="Parthenium hysterophorus"

In [3]:
# creating request for flora online

# http://www.worldfloraonline.org/search?query=parthenium+hysterophorus&view=&limit=2000&start=0&sort=

url_base = "http://www.worldfloraonline.org"



In [4]:
def wfo_url_builder(plant_name):
    
    """ builds a link based on the plant name fit for world flora online"""
    
    url_base = "http://www.worldfloraonline.org"
    clean_plant_name=plant_name.replace(" ", "+")
    return url_base + "/search?query="+clean_plant_name.lower()
    
    
def send_request(complete_url):
    """ sends request and parses response data,
        requires beautifulSoup and complete url made with wof_url_builder 
        returns 3 items, the 3rd is the content    """

    response = requests.get(complete_url) # TODO: add timeout for the request
    soup = BeautifulSoup(response.content, "html.parser")
    return response.url, response.status_code, soup 

In [5]:
# function to search for good links from the list of plants usese status == "Accepted Name" and plant name to select good link

def get_good_link_wfo(soup):
    """function to search for good links from the list of plants usese status == "Accepted Name" 
    and plant name to select good link"""
    all_plants = soup.findAll("td")

    for plant in all_plants:

        try:
            title = plant.find("a", class_="result").get("title")
            status = plant.find("span").text
            link = url_base + plant.find("a", class_="result").get("href")
        except AttributeError:
            continue

        if (status == "Accepted Name") & (title == plant_name):
            break
    return link


In [6]:
def get_text_from_section(soup, lv1_tag, kwargs) :
    """extract text from specified section. returns list of paragraphs
    params: 
    - soup : content of request
    -lv1_tag : HTML tag fror area with text
    -kwargs : dictionary with attribute name and value to use
    - kwargs:"""
    gen = soup.find(lv1_tag, kwargs)
    text_list=[]
    for n,a in enumerate(gen):
        try:
            text_list.append(a.p.text)   

        except AttributeError:
            continue

    return text_list    

### Settings

In [7]:
# plant name will eventually be an input from csv file

plant_name="Parthenium hysterophorus"
# creating request for flora online

# http://www.worldfloraonline.org/search?query=parthenium+hysterophorus&view=&limit=2000&start=0&sort=

url_base = "http://www.worldfloraonline.org"

sections=["general", "morphology", "habit", "habitat"]

lv1_tag = "div"
lv1_atr_tag = 'id'


### Main function to scrape flora online

In [8]:
# running functions
def main_flora_online(plant_name, url_base, sections, lv1_tag, lv1_atr_tag):
    complete_url = wfo_url_builder(plant_name)

    url, status, soup = send_request(complete_url)

    print("main page : ", url, status)

    link = get_good_link_wfo(soup)

    url2, status2, soup2 = send_request(link)
    print("plant page :", url2, status2)  # TODO add failsafe in case link doesn't work

    # get text for specified section
    # def get_sections(soup, sections):
    text_dic={}
    for i in sections:
        lv1_atr_value = i
        text_dic[i] = get_text_from_section(soup2, lv1_tag, { lv1_atr_tag: lv1_atr_value})
    return text_dic



# Create a summary  from scraped text

1. download and install bert summariser or similar
1. Select text to send
1. Get summary

###  Notes
possible summarizers:
- https://pypi.org/project/bert-extractive-summarizer/
- https://smrzr.io/
--


In [9]:
Text_list = main_flora_online(plant_name, url_base, sections, lv1_tag, lv1_atr_tag)

main page :  http://www.worldfloraonline.org/search?query=parthenium+hysterophorus 200
plant page : http://www.worldfloraonline.org/taxon/wfo-0000065156;jsessionid=555BDFCA5AC35C6C605AADF418C54332 200


In [10]:
from summarizer import Summarizer

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer()
model(body)
model(body2)

ModuleNotFoundError: No module named 'summarizer'